In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import confusion_matrix, precision_score, recall_score

2022-12-13 22:57:13.127265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 22:57:13.428923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-13 22:57:13.428944: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-13 22:57:13.470390: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-13 22:57:14.301212: W tensorflow/stream_executor/platform/de

In [2]:
df = pd.read_csv("dades/points_data.csv")

In [3]:
# Select quantitatives variables
quant_var = df[["altitut", "pendent", "orientacio", "t_max", "u", "v", "specific_humidity", "relative_humidity"]]
# Normalize
mean = quant_var.mean()
mean.to_csv("dades/means.csv")
std = quant_var.std()
std.to_csv("dades/stds.csv")
normalized_df = (quant_var-quant_var.mean())/quant_var.std()
# Add soil use binarizaded
normalized_df = normalized_df.join(pd.get_dummies(df["coberta_sol"].astype("category")))

In [4]:
# Split dataset
y = df["fire"]
xTrain, xTest, yTrain, yTest = train_test_split(normalized_df, y, test_size=0.2, random_state=12)

In [5]:
# Design neural net
model = keras.Sequential()

model.add(keras.layers.Conv1D(64, kernel_size=1, activation='relu', input_shape=(19,1)))
model.add(keras.layers.Dropout(0.2))

    
model.add(keras.layers.Conv1D(128, 3, activation='relu'))
model.add(keras.layers.Dropout(0.2))

   
model.add(keras.layers.Conv1D(256, 3, activation='relu'))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))

model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

2022-12-13 22:57:15.360056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-13 22:57:15.360188: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-13 22:57:15.360204: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xpasku-PROX15-AMD): /proc/driver/nvidia/version does not exist
2022-12-13 22:57:15.360423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=20)

# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("Confusion matrix:", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Epoch 1/20
197/197 [==============================] - 2s 5ms/step - loss: 0.2624 - accuracy: 0.8963
Epoch 2/20
197/197 [==============================] - 1s 5ms/step - loss: 0.2186 - accuracy: 0.9156
Epoch 3/20
197/197 [==============================] - 1s 5ms/step - loss: 0.2128 - accuracy: 0.9183
Epoch 4/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1985 - accuracy: 0.9212
Epoch 5/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1977 - accuracy: 0.9210
Epoch 6/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1904 - accuracy: 0.9247
Epoch 7/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1847 - accuracy: 0.9254
Epoch 8/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1823 - accuracy: 0.9250
Epoch 9/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1782 - accuracy: 0.9286
Epoch 10/20
197/197 [==============================] - 1s 5ms/step - loss: 0.1737 - accuracy: 0.9308

In [7]:
# Export model
model.save("dades/nn_fire_risk.h5")